### Demonstration of an evaluation based on the MCTSAgent

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload

In [ ]:
import chess
import chess.variant
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.insert(0,'../../../')
import DeepCrazyhouse.src.runtime.Colorer
from DeepCrazyhouse.src.domain.agent.NeuralNetAPI import NeuralNetAPI
from DeepCrazyhouse.src.domain.agent.player.MCTSAgent import MCTSAgent
from DeepCrazyhouse.src.domain.agent.player.RawNetAgent import RawNetAgent
from DeepCrazyhouse.src.domain.crazyhouse.GameState import GameState
from time import time
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [ ]:
batch_size = 8 #1 #8

In [ ]:
net = NeuralNetAPI(ctx='gpu', batch_size=batch_size)

In [ ]:
raw_agent = RawNetAgent(net)

In [ ]:
mcts_agent = MCTSAgent(net, threads=16, playouts_empty_pockets=4096, playouts_filled_pockets=4096,
                 cpuct=2, dirichlet_epsilon=.1, dirichlet_alpha=0.2, batch_size=batch_size, q_value_weight=0.,
                 max_search_depth=40, temperature=0., clip_quantil=0., virtual_loss=3, verbose=False, min_movetime=10000)

In [ ]:
board = chess.variant.CrazyhouseBoard()

#board.push_uci('e2e4')
#board.push_uci('e7e6')

#fen = 'rnbqkb1r/ppp1pppp/5n2/3P4/8/8/PPPP1PPP/RNBQKBNR/P w KQkq - 1 3'
#fen = 'r4rk1/ppp2pp1/3p1q1p/n1bPp3/2B1B1b1/3P1N2/PPP2PPP/R2Q1RK1[Nn] w - - 2 13'
#fen = 'rnb2rk1/p3bppp/2p5/3p2P1/4n3/8/PPPPBPPP/RNB1K1NR/QPPq w KQ - 0 11'
#fen = 'r1b1kbnr/ppp1pppp/2n5/3q4/3P4/8/PPP1NPPP/RNBQKB1R/Pp b KQkq - 1 4'
#fen = 'r1b1k2r/ppp2ppp/2n5/3np3/3P4/2PBP3/PpPB1PPP/1Q2K1NR/QNrb b Kkq - 27 14'
#fen = 'r1bb4/ppp2pkp/5npb/4p3/4P3/2N5/PPP1BPPP/3RK2R/QNRqpnp w K - 3 16'
#fen = '3R1b2/1bP1kp2/3Npn1p/3p4/5p2/5N1b/PPP2P1P/5Q1K/RPRPRQPnpbp b - - 4 36'
#fen = 'r3k1nr/ppp2ppp/2n5/8/q7/Pp6/1PnRN1PP/1K3B1R/BBQBppppp b kq - 3 27'
#fen = 'rn2N2k/pp5p/3pp1pN/3p4/3q1P2/3P1p2/PP3PPP/RN3RK1/Qrbbpbb b - - 0 27'
#fen = 'rn2N2k/pp5p/3pp1pN/3p4/3q1P2/3P1p2/PP3PPP/RN3RK1/Qrbbpbb b - - 0 30'
#fen = 'rn2N2k/pp5p/3pp1pN/3p4/5P2/3P1p2/PP3RPP/RN4K1/QQprbbpbb b - - 1 30'
#fen = 'rn2N1bk/pp5p/3pp1pN/3p4/3q1P2/3P1p2/PP1N1PPP/R4RK1/Qrbpbb b - - 2 30'

#fen = '3R1b2/1bP1kp2/3Npn1p/3p4/5p2/5N1b/PPP1QP1P/3R1RK1/QPpprnpbp b - - 0 29'
#fen = 'rn2N2k/pp5p/3pp1pN/3p4/3q1P2/3P1p2/PP3PPP/RN3RK1[Qrbbpbb] b - - 3 30'
#fen = 'r4r1k/ppp1q1bp/4Ppp1/1P6/2NN3P/2BPPb2/P2PNPpR/R3K3/PBQn b Q - 2 34'
#fen = 'r1bq1b1r/ppp1kPpp/4Pn2/n2Pp3/2B4n/3P4/PPP2PPP/RNBQK2R/ w KQ - 0 10'

#fen = 'r4rk1/ppp2pbp/1nnp4/6P1/4Pp2/1P1P1N1P/2P2PPK/R2Q1R2/QNbbpb w - - 0 23'
#fen = 'r4rk1/ppp2pPp/1nn3nP/4b3/5p1B/1P1P3P/2P2PPK/R2Q1R2/Qbpnpb w - - 2 37'
#fen = 'r4rk1/ppp2pPp/1nn2bnP/4b3/6Q1/1P1P1pBP/2P2PPK/R4R2/Qpnpb w - - 2 39'
#fen = 'B2q1r1k/p3pp2/6pb/3pB2p/3N4/B2NP1pn/P1PP2P1/R2Q3K/PPrnprp b - - 0 28'
#fen = 'r1bq3r/p1ppbkpp/2p2n2/8/4P3/8/PPP2PPP/RNBQK2R/NPPnbp w KQ - 2 8'
#fen = 'r1bqk2r/1pp1nppp/1p3p2/3p4/3P3n/1B6/PP2BPPP/RNBQ1KNR/Pp b kq - 0 14'
#fen = '3qk2r/1pp1nppp/1p3p2/3p4/3P3n/1B5P/RP2BPpP/1NBQ2KR/RBPPn b k - 2 18'
#fen = 'r3q1r1/1Np1p1bk/1p3ppr/pP2p3/4P1KQ/4PP1N/P1PP4/2r2n1q~/bbbnp b - - 0 49'
#fen = 'r3q1r1/1Np1p1bk/1p3ppr/pP2p3/4P2Q/4PPKN/P1PP4/2r4q~/bbbnnp b - - 0 48'
#fen = 'r2q2kb/ppp4p/1nn1P1p1/4p1N1/2Pb4/1P1P2NP/2PB1rP1/Q4RK1/RPppb w - - 0 27'
fen = 'r1bqk1r1/ppppbppp/5n2/4nN2/2B1P3/2N5/PPP2PPP/R1BQK2R/Pp w KQq - 2 8'
board.set_fen(fen)
#board = board.mirror()

state = GameState(board)
board

In [ ]:
len(list(state.get_legal_moves()))

In [ ]:
def plot_moves_with_prob(moves, probs, only_top_x=None):
    
    # revert the ordering afterwards
    idx_order = np.argsort(probs)[::-1]
    
    if only_top_x is not None and only_top_x < len(idx_order):
        idx_order = idx_order[:only_top_x]
    
    #moves_ordered = moves[range(len(moves))] #idx_order[::-1]]
    probs_ordered = [] #probs[idx_order]
    
    moves_ordered = []
    for idx in idx_order:
        probs_ordered.append(probs[idx])
        moves_ordered.append(moves[idx])
        
    plt.barh(range(len(probs_ordered)), probs_ordered)
    plt.yticks(range(len(moves_ordered)), moves_ordered)


### Evalution using the raw network

In [ ]:
t_s = time()
value, legal_moves, p_vec_small = raw_agent.evaluate_board_state(state)
print('Elapsed time: %.4fs' % (time()-t_s))

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

### Evalution using the MCTS-Agent

In [ ]:
t_s = time()
value, legal_moves, p_vec_small = mcts_agent.evaluate_board_state(state)
print('Elapsed time: %.4fs' % (time()-t_s))

In [ ]:
list(board.legal_moves)[mcts_agent.root_node.q.argmax()]

In [ ]:
mcts_agent.get_calculated_line()

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
mcts_agent._run_single_playout(state, mcts_agent.root_node)